In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Customer_details='''

--DECLARE _end_dt_snpsht_tmp DATE DEFAULT CURRENT_DATE();
DECLARE _end_dt_snpsht_tmp DATE DEFAULT '2023-02-04';

WITH date_sql as (

SELECT 

last_day(date_sub(_end_dt_snpsht_tmp, interval 1 month), month) as last_date_prev_month
,date_trunc(date_sub(_end_dt_snpsht_tmp, interval 1 month), month) as first_date_prev_month

)



, ADC_Customer_Base as (

SELECT date((select first_date_prev_month from date_sql)) as Month_Snapshot,customer_id,dealer_customer_id,primary_login_id,dealer_name,join_date,account_type_name,customer_type_name,primary_email,primary_phone,last_updt_ts
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
where date(last_updt_ts) =(select last_date_prev_month from date_sql)
and  dealer_name in ('TELUS Communications Inc.','ADT by TELUS')
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1
order by dealer_name,join_date,customer_id



)


, ADT_Customer_Base as (

SELECT customer_id,'ADT_Customer' as Dealer_type_flag,NULL  as BAN,site_no as ADT_site_no 
FROM `divgpras-pr-579355.ADC_updated.ADC_ADT_mapping` 

)


/*
,Telus_customers as
(
select bacct_bus_bacct_num as BAN,cust_bus_cust_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,prod_instnc_ts,pi_cntrct_start_ts as contract_start_date,pi_cntrct_end_ts as contract_end_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = (select last_date_prev_month from date_sql) #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
order by cust_bus_cust_id

)

*/

,Telus_customers as
(
select bacct_bus_bacct_num,bacct_billg_acct_id,cust_bus_cust_id
--,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,prod_instnc_ts,pi_cntrct_start_ts as contract_start_date
--,pi_cntrct_end_ts as contract_end_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = (select last_date_prev_month from date_sql) #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
--QUALIFY ROW_NUMBER() OVER (PARTITION BY bacct_bus_bacct_num ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by cust_bus_cust_id

)



 ,Telus_Customer_Base as (

SELECT a.customer_id,'Telus_Customer' as Dealer_type_flag,b.bacct_bus_bacct_num as BAN, NULL as ADT_site_no
FROM ADC_Customer_Base a
inner join Telus_customers b
on a.dealer_customer_id=b.cust_bus_cust_id
)



, Union_data as (

SELECT * from Telus_Customer_Base
UNION ALL
SELECT * from ADT_Customer_Base

)

,Merge_data as (

SELECT * from ADC_Customer_Base a
INNER JOIN Union_data b

on a.customer_id=b.customer_id
order by Dealer_type_flag
)


/*
 ,Merge_data as (

SELECT *
FROM ADC_Customer_Base a
left join Telus_customers b
on a.dealer_customer_id=b.cust_bus_cust_id

)
*/



select * from Merge_data



'''

In [ ]:
Customer_info=extract_bq_data(bq_client, sql=Customer_details)

In [ ]:
Customer_info.info()

In [ ]:
Customer_info.head()

In [ ]:
Best_practices_details_Query='''

--DECLARE _end_dt_snpsht_tmp DATE DEFAULT CURRENT_DATE();
DECLARE _end_dt_snpsht_tmp DATE DEFAULT '2023-02-04';

WITH date_sql as (

SELECT 

last_day(date_sub(_end_dt_snpsht_tmp, interval 1 month), month) as last_date_prev_month
,date_trunc(date_sub(_end_dt_snpsht_tmp, interval 1 month), month) as first_date_prev_month

)

select customer_id,best_practices_id,best_practices_ind,date(dt_last_calculate_utc) as last_date_calculate
from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_best_practice`
where DATE(dt_last_calculate_utc) < (select last_date_prev_month from date_sql)
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id,best_practices_id ORDER BY last_updt_ts DESC) = 1
order by last_date_calculate


'''

In [ ]:
Best_Practices_DF= extract_bq_data(bq_client,sql=Best_practices_details_Query)

In [ ]:
Best_Practices_DF.info()

In [ ]:
Best_Practices_DF.head()

In [ ]:
Best_Practices_DF['last_date_calculate'].max()

In [ ]:
Best_Practices_DF['Best_practices_flag']=Best_Practices_DF['best_practices_ind'].apply(lambda x: 1 if x==True else 0)
Best_Practices_DF_wide=Best_Practices_DF.pivot(index='customer_id', columns='best_practices_id', values='Best_practices_flag').reset_index()
Best_Practices_DF_wide.columns=['customer_id','Best_partices_1','Best_partices_2','Best_partices_3','Best_partices_4','Best_partices_5','Best_partices_6']
cols_to_sum=['Best_partices_1','Best_partices_2','Best_partices_3','Best_partices_4','Best_partices_5','Best_partices_6']
Best_Practices_DF_wide['Best_Practice_All']=Best_Practices_DF_wide[cols_to_sum].sum(axis=1)
Best_Practices_DF_wide['Best_Practice_All_flag']=Best_Practices_DF_wide['Best_Practice_All'].apply(lambda x: 1 if x==6 else 0)

In [ ]:
Best_Practices_DF_wide.head()

In [ ]:
Arming_Query=  '''

--DECLARE _end_dt_snpsht_tmp DATE DEFAULT CURRENT_DATE();
DECLARE _end_dt_snpsht_tmp DATE DEFAULT '2023-02-04';

WITH date_sql as (

SELECT 

last_day(date_sub(_end_dt_snpsht_tmp, interval 1 month), month) as last_date_prev_month
,date_trunc(date_sub(_end_dt_snpsht_tmp, interval 1 month), month) as first_date_prev_month

)


, date_sql_arming as (
select  count(distinct date(date)) as count_of_dates from `cio-datahub-enterprise-pr-183a.src_adc.bq_aggregate_daily_arming_commands`
where date(date)< (select last_date_prev_month from date_sql)
and date(date)>= (select first_date_prev_month from date_sql)

)

select id_cust as customer_id,sum(count_arm_commands) as sum_arm_commands,sum(count_disarm_commands) as sum_disarm_commands,count (distinct date(date)) as number_days_arming_disarming, (select count_of_dates from date_sql_arming ) as count_of_dates_arming
from `cio-datahub-enterprise-pr-183a.src_adc.bq_aggregate_daily_arming_commands`
where date(date)< (select last_date_prev_month from date_sql)
and date(date)>= (select first_date_prev_month from date_sql)
group by customer_id
order by number_days_arming_disarming desc

'''

In [ ]:
Arming_DF=extract_bq_data(bq_client, sql=Arming_Query)

In [ ]:
Arming_DF.info()

In [ ]:
Arming_DF.head()

In [ ]:
max_number_arming_dates=Arming_DF['count_of_dates_arming'].max()
Arming_DF['Arming_Consistency']=Arming_DF['number_days_arming_disarming'].apply(lambda x: x*100/max_number_arming_dates)

In [ ]:
Login_SQL='''


--DECLARE _end_dt_snpsht_tmp DATE DEFAULT CURRENT_DATE();
DECLARE _end_dt_snpsht_tmp DATE DEFAULT '2023-02-04';

WITH date_sql as (

SELECT 

last_day(date_sub(_end_dt_snpsht_tmp, interval 1 month), month) as last_date_prev_month
,date_trunc(date_sub(_end_dt_snpsht_tmp, interval 1 month), month) as first_date_prev_month

)


, date_sql_login as (
select  count(distinct date(login_dt_utc)) as count_of_dates from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)< (select last_date_prev_month from date_sql)
and date(login_dt_utc)>= (select first_date_prev_month from date_sql)

)



select customer_id,sum(login_count) as sum_login_count,count (distinct date(login_dt_utc)) as number_of_login_days, (select count_of_dates from date_sql_login ) as count_of_dates_logins
from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)< (select last_date_prev_month from date_sql)
and date(login_dt_utc)>= (select first_date_prev_month from date_sql) 
group by customer_id
order by number_of_login_days desc


'''

In [ ]:
Login_DF=extract_bq_data(bq_client, sql=Login_SQL)

In [ ]:
Login_DF.info()

In [ ]:
Login_DF.head()

In [ ]:
max_number_login_dates=Login_DF['count_of_dates_logins'].max()
Login_DF['Login_Consistency']=Login_DF['number_of_login_days'].apply(lambda x: x*100/max_number_login_dates)

In [ ]:
Login_DF.head()

In [ ]:
TC_issue_query='''

--DECLARE _end_dt_snpsht_tmp DATE DEFAULT CURRENT_DATE();
DECLARE _end_dt_snpsht_tmp DATE DEFAULT '2023-02-04';

WITH date_sql as (

SELECT 

last_day(date_sub(_end_dt_snpsht_tmp, interval 1 month), month) as last_date_prev_month
,date_trunc(date_sub(_end_dt_snpsht_tmp, interval 1 month), month) as first_date_prev_month
,date_trunc(date_sub(_end_dt_snpsht_tmp, interval 4 month), month) as first_date_prev_3_month

)


select dealer_name,customer_id,unit_id,device_id,trouble_condition_name,trouble_condition_group_id,trouble_condition_group_desc, start_date_utc,end_date_utc
from `cio-datahub-enterprise-pr-183a.src_adc.bq_troublecondition_data`
where trouble_condition_start_ind='Y'and trouble_condition_closed_ind='Y' 
and date(start_date_utc) >= (select first_date_prev_3_month from date_sql) 
and date(start_date_utc) <(select first_date_prev_month from date_sql)


'''

In [ ]:
TC_data= extract_bq_data(bq_client, sql=TC_issue_query)

In [ ]:
TC_data.head()

In [ ]:
TC_data['trouble_condition_name'].nunique()

In [ ]:
TC_data['start_date_utc'].min()

In [ ]:
TC_data['start_date_utc'].max()

In [ ]:
TC_data['count']=1

TC_pivot_table=pd.pivot_table(TC_data, values=['count'], index=['customer_id'], columns='trouble_condition_name',
                          aggfunc={'count': 'sum'})

TC_pivot_table=TC_pivot_table.reset_index()

TC_pivot_table.fillna(0,inplace=True)

In [ ]:
TC_pivot_table.columns=['TC_Last3M_'+'_'.join(col).strip() for col in TC_pivot_table.columns.values]

In [ ]:
# TC_pivot_cols_required=TC_pivot_table.columns.to_list()

In [ ]:
# TC_pivot_cols_required

In [ ]:
# import yaml
# with open(r'/home/jupyter/ADC/notebooks/Feature_data_store/TC_pivot_cols_required.yaml', 'w') as file:
#     documents = yaml.dump(TC_pivot_cols_required, file)

# import json
    
# with open(r'/home/jupyter/ADC/notebooks/Feature_data_store/TC_pivot_cols_required.json', 'w') as Ffp:
#     json.dump(TC_pivot_cols_required, Ffp)


In [ ]:
TC_pivot_table.head()

In [ ]:
def add_missing_dummy_columns(d, cols_req_model):
    ''' 
    This function adds missing columns to the data so that the model doesn't fail.
    d is the Dataset. 
    cols_req_model is the list of columns required by the model.
    
    '''
    missing_cols = list(set(cols_req_model) - set(d.columns))
    print('Adding columns', missing_cols, 'to the data and populating them with 0.')
    for c in missing_cols:
        d[c] = 0

In [ ]:
import json
with open(r'/home/jupyter/ADC/notebooks/Feature_data_store/TC_pivot_cols_required.json') as json_file:
    final_feature_cols_TC = json.load(json_file)

In [ ]:
# final_feature_cols_TC

In [ ]:
add_missing_dummy_columns(TC_pivot_table,final_feature_cols_TC)

In [ ]:
def remove_extra_columns(d,cols_req_model):
    
    ''' 
    This function drops extra columns from the data so that the model doesn't fail.
    d is the Dataset. 
    cols_req_model is the list of columns required by the model.
    
    '''
    extra_cols= list(set( d.columns ) - set( cols_req_model ))
    print('Dropping', extra_cols, 'columns from the data.')
    d = d.drop(extra_cols, axis=1)
    return d

In [ ]:
TC_pivot_table=remove_extra_columns(TC_pivot_table,final_feature_cols_TC)

In [ ]:
TCG_Pivot=pd.pivot_table(TC_data, values=['count'], index=['customer_id'], columns='trouble_condition_group_desc',
                          aggfunc={'count': np.sum})
TCG_Pivot=TCG_Pivot.reset_index()
TCG_Pivot.fillna(0,inplace=True)
TCG_Pivot.columns=['TCG_Last3M_'+'_'.join(col).strip() for col in TCG_Pivot.columns.values]

In [ ]:
TCG_Pivot.head()

In [ ]:
Merge_DF=Customer_info.merge(Best_Practices_DF_wide,on='customer_id',how='left')

In [ ]:
Merge_DF_1=Merge_DF.merge(Arming_DF,on='customer_id',how='left')
Merge_DF_2=Merge_DF_1.merge(Login_DF,on='customer_id',how='left')
Merge_DF_3=Merge_DF_2.merge(TC_pivot_table,left_on='customer_id',right_on='TC_Last3M_customer_id_',how='left')
Merge_DF_4=Merge_DF_3.merge(TCG_Pivot,left_on='customer_id',right_on='TCG_Last3M_customer_id_',how='left')

In [ ]:
del Merge_DF_1,Merge_DF_2,Merge_DF_3

In [ ]:
rep_chars = ' |\|-|:|/'

Merge_DF_4.columns = Merge_DF_4.columns.str.replace(rep_chars, '_')

In [ ]:
def Segment_making(row
                   ,min_login_consistency_threshold=50
                   ,min_arming_consistency_threshold=50
                   ,max_login_consistency_threshold=50
                   ,max_arming_consistency_threshold=50):
    
    if row['Best_Practice_All']>=5 and row['Login_Consistency']>=max_login_consistency_threshold  and row['Arming_Consistency']>=max_arming_consistency_threshold:
        return 'Heavy_User'
    elif row['Login_Consistency']>=max_login_consistency_threshold  and row['Arming_Consistency']< min_arming_consistency_threshold:
        return 'Home_automation_Savvy'
    elif row['Login_Consistency']< min_login_consistency_threshold  and row['Arming_Consistency']>=max_login_consistency_threshold:
        return 'Old_Fashion'
    elif row['Login_Consistency']==0  and row['Arming_Consistency']==0:
        return 'Disengaged'
    else:
        return "Moderate_Users"

In [ ]:
Merge_DF_4['Segment']=Merge_DF_4.apply(Segment_making,axis=1)

In [ ]:
Merge_DF_4['Segment'].value_counts(normalize=True)*100

In [ ]:
Merge_DF_4.info(verbose=True)

In [ ]:
Merge_DF_4.head()

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

Table_BQ = 'ADC_Feature_Datastore.ADC_Master_Data'

bq_table_instance= bq_client.load_table_from_dataframe(Merge_DF_4, Table_BQ,job_config=config)

In [ ]:
Query_test='''

declare end_date date;

set end_date = DATE('{score_date}');



WITH date_sql as (

SELECT 

last_day(date_sub(end_date, interval 1 month), month) as last_date_prev_month
,date_trunc(date_sub(end_date, interval 1 month), month) as first_date_prev_month

)


, date_sql_arming as (
select  count(distinct date(date)) as count_of_dates from `cio-datahub-enterprise-pr-183a.src_adc.bq_aggregate_daily_arming_commands`
where date(date)< (select last_date_prev_month from date_sql)
and date(date)>= (select first_date_prev_month from date_sql)

)

select id_cust as customer_id,sum(count_arm_commands) as sum_arm_commands,sum(count_disarm_commands) as sum_disarm_commands,count (distinct date(date)) as number_days_arming_disarming, (select count_of_dates from date_sql_arming ) as count_of_dates_arming
from `cio-datahub-enterprise-pr-183a.src_adc.bq_aggregate_daily_arming_commands`
where date(date)< (select last_date_prev_month from date_sql)
and date(date)>= (select first_date_prev_month from date_sql)
group by customer_id
order by number_days_arming_disarming desc


'''

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
from datetime import date
SCORE_DATE = date.today().strftime('%Y-%m-%d')

In [ ]:
SCORE_DATE

In [ ]:
# DF=extract_bq_data(bq_client, sql=Query_test).format(score_date='2023-01-26')